In [2]:
!pip install pyyaml h5py

In [3]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.7.0


In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [8]:
# Define a simple sequential model
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.metrics.SparseCategoricalAccuracy()])
    return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [9]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

Epoch 1/10
24/32 [=====================>........] - ETA: 0s - loss: 1.2639 - sparse_categorical_accuracy: 0.6484
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 15ms/step - loss: 1.1445 - sparse_categorical_accuracy: 0.6800 - val_loss: 0.6971 - val_sparse_categorical_accuracy: 0.8020
Epoch 2/10
30/32 [===========================>..] - ETA: 0s - loss: 0.4309 - sparse_categorical_accuracy: 0.8771
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 0.4260 - sparse_categorical_accuracy: 0.8800 - val_loss: 0.5199 - val_sparse_categorical_accuracy: 0.8520
Epoch 3/10
27/32 [========================>.....] - ETA: 0s - loss: 0.2920 - sparse_categorical_accuracy: 0.9248
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 0.2811 - sparse_categorical_accuracy: 0.9270 - val_loss: 0.4699 - val_sparse_categorical_accuracy: 0.8520
Epoch 4/10
25/3

In [10]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_1/cp.ckpt'

In [11]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3290 - sparse_categorical_accuracy: 0.1140 - 219ms/epoch - 7ms/step
Untrained model, accuracy: 11.40%


In [12]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4279 - sparse_categorical_accuracy: 0.8630 - 72ms/epoch - 2ms/step
Restored model, accuracy: 86.30%


In [14]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [15]:
os.listdir(checkpoint_dir)

['cp-0005.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index']

In [16]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [17]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4938 - sparse_categorical_accuracy: 0.8720 - 185ms/epoch - 6ms/step
Restored model, accuracy: 87.20%


In [18]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4938 - sparse_categorical_accuracy: 0.8720 - 166ms/epoch - 5ms/step
Restored model, accuracy: 87.20%


In [19]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1343 - sparse_categorical_accuracy: 0.6760
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4400 - sparse_categorical_accuracy: 0.8750
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2926 - sparse_categorical_accuracy: 0.9190
Epoch 4/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2149 - sparse_categorical_accuracy: 0.9410
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1638 - sparse_categorical_accuracy: 0.9660
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [21]:
# my_model directory
!ls saved_model

my_model


In [22]:
# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

assets            keras_metadata.pb saved_model.pb    variables


In [23]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4499 - sparse_categorical_accuracy: 0.8470 - 187ms/epoch - 6ms/step
Restored model, accuracy: 84.70%
(1000, 10)


In [25]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1431 - sparse_categorical_accuracy: 0.6900
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4234 - sparse_categorical_accuracy: 0.8760
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2896 - sparse_categorical_accuracy: 0.9280
Epoch 4/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2018 - sparse_categorical_accuracy: 0.9520
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1590 - sparse_categorical_accuracy: 0.9640


In [26]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 512)               401920    
                                                                 
 dropout_8 (Dropout)         (None, 512)               0         
                                                                 
 dense_17 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
